## Practice notebook for confidence intervals using NHANES data

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as st

da = pd.read_csv("nhanes_2015_2016.csv")

### Question 1
Restrict the sample to women between 35 and 50, then use the marital status variable DMDMARTL to partition this sample into two groups - women who are currently married, and women who are not currently married. Within each of these groups, calculate the proportion of women who have completed college. Calculate 95% confidence intervals for each of these proportions.

In [2]:
da['agegrp'] = pd.cut(da.RIDAGEYR, [35, 50])
da['RIAGENDRx'] = da.RIAGENDR.replace({1:'Male', 2:'Female'})
da['DMDMARTLx'] = da.DMDMARTL.replace({1:"Married", 2:"Not Married", 3:"Not Married", 4:"Not Married",
                                      5:"Not Married", 6:"Not Married", 77:"na"}).dropna()
da['DMDEDUC2x'] = da.DMDEDUC2.replace({1: "<9", 2: "9-11", 3: "HS/GED", 4: "Some college/AA", 5: "College", 
                                       7: "Refused", 9: "Don't know"})

In [3]:
# Within each of these groups, calculate the proportion of women who have completed college.
dx = da.groupby(['RIAGENDRx', 'agegrp', 'DMDMARTLx']).DMDEDUC2x.value_counts().unstack()
dx = dx.apply(lambda z: z / z.sum(), axis = 1)
dx

DMDEDUC2x                           9-11        <9   College    HS/GED  \
RIAGENDRx agegrp   DMDMARTLx                                             
Female    (35, 50] Married      0.095465  0.093079  0.367542  0.140811   
                   Not Married  0.109034  0.121495  0.208723  0.205607   
Male      (35, 50] Married      0.120000  0.115000  0.307500  0.195000   
                   Not Married  0.194313  0.118483  0.137441  0.270142   
                   na                NaN       NaN       NaN       NaN   

DMDEDUC2x                       Some college/AA  
RIAGENDRx agegrp   DMDMARTLx                     
Female    (35, 50] Married             0.303103  
                   Not Married         0.355140  
Male      (35, 50] Married             0.262500  
                   Not Married         0.279621  
                   na                  1.000000

In [4]:
x = da[(da.RIAGENDRx == 'Female') & (da.agegrp == pd.Interval(35, 50)) & (da.DMDEDUC2x == 'College')].DMDMARTLx.value_counts()
y = da[da.DMDEDUC2x == 'College'].DMDMARTLx.value_counts().sum()
freq = x / y
freq

Married        0.112738
Not Married    0.049048
Name: DMDMARTLx, dtype: float64

In [5]:
dx = da.groupby(['RIAGENDRx', 'agegrp', 'DMDMARTLx']).agg({'DMDEDUC2x':np.size}).unstack()
print(dx)

                   DMDEDUC2x                 
DMDMARTLx            Married Not Married   na
RIAGENDRx agegrp                             
Female    (35, 50]     419.0       321.0  NaN
Male      (35, 50]     400.0       211.0  1.0


In [6]:
# Calculate 95% confidence intervals for married.
p_married = 0.367542
n = 419
se_married = np.sqrt(p_married * (1 - p_married)/ n)

lcb = p_married - 1.96 * se_married
ucb = p_married + 1.96 * se_married
(lcb, ucb)

(0.32137640979550464, 0.4137075902044953)

In [7]:
# Calculate 95% confidence intervals for not married.
p_married = 0.208723
n = 321
se_married = np.sqrt(p_married * (1 - p_married)/ n)

lcb = p_married - 1.96 * se_married
ucb = p_married + 1.96 * se_married
(lcb, ucb)

(0.1642646868597602, 0.2531813131402398)

### Question 2
Construct a 95% confidence interval for the proportion of smokers who are female. Construct a 95% confidence interval for the proportion of smokers who are male. Construct a 95% confidence interval for the difference between those two gender proportions.

In [8]:
da['SMQ020x'] = da.SMQ020.replace({1:'Yes', 2:'No', 7:np.nan, 9:np.nan})
da['SMQ020x'].describe()

count     5725
unique       2
top         No
freq      3406
Name: SMQ020x, dtype: object

In [9]:
dx = da[['SMQ020x', 'RIAGENDRx']].dropna()
pd.crosstab(dx.SMQ020x, dx.RIAGENDRx)

RIAGENDRx,Female,Male
SMQ020x,,
No,2066,1340
Yes,906,1413


In [10]:
dx["SMQ020x"] = dx.SMQ020x.replace({"Yes":1, "No":0})

In [11]:
dz = dx.groupby(['RIAGENDRx']).agg({'SMQ020x':[np.mean, np.size]})
dz.columns = ['Proportion', 'Total n']
dz

,Proportion,Total n
RIAGENDRx,,
Female,0.304845,2972
Male,0.513258,2753


In [12]:
# Construct a 95% confidence interval for the proportion of smokers who are female
p = 0.304845
n = 2972
se_female = np.sqrt(p * (1 - p) / n)
lcb = p - 1.96 * se_female
ucb = p + 1.96 * se_female
(lcb, ucb)

(0.2882944651781637, 0.32139553482183625)

In [13]:
# Construct a 95% confidence interval for the proportion of smokers who are male
p = .513258
n = 2753
se_male = np.sqrt(p * (1 - p)/ n)
lcb = p - 1.96 * se_male
ucb = p + 1.96 * se_male
(lcb, ucb)

(0.49458688557746244, 0.5319291144225375)

In [14]:
# Construct a 95% confidence interval for the difference between those two gender proportions.
se_diff = np.sqrt(se_female ** 2 + se_male ** 2)
d = 0.304845 - 0.513258
lcb = d - 1.96 * se_diff
ucb = d + 1.96 * se_diff
(lcb, ucb)

(-0.23336356545788706, -0.18346243454211297)

### Question 3
Construct a 95% confidential interval for height (BMXHT) in centimeters. Then convert height from centimeters to inches by dividing by 2.54, and construct a 95% confidence interval for height in inches. Finally, convert the endpoints (the lower and upper confidence limits) of the confidence interval from inches to back to centimeters

In [15]:
# Construct a 95% confidential interval for height in cm
mean = da["BMXHT"].mean()
print("mean = ", mean)

sd = da["BMXHT"].std()
print("std = ", sd)

n = len(da)
print(n)

mean =  166.14283447911131
std =  10.079263712467688
5735


In [16]:
se = sd / np.sqrt(n)
lcb = mean - 1.96 * se
ucb = mean + 1.96 * se
(lcb, ucb)

(165.88196806498644, 166.4037008932362)

In [17]:
da["BMXHT"].head()

0    184.5
1    171.4
2    170.1
3    160.9
4    164.9
Name: BMXHT, dtype: float64

In [18]:
# convert height from cm to inches
da["BMXHTin"] = da.BMXHT / 2.54
da["BMXHTin"].head()

0    72.637795
1    67.480315
2    66.968504
3    63.346457
4    64.921260
Name: BMXHTin, dtype: float64

In [19]:
# construct a 95% confidence interval for height in inches
mean = da["BMXHTin"].mean()
print("mean = %s" % (mean))
sd = da["BMXHTin"].std()
print("std = %s" % sd)
n = len(da)
print(n)
se = sd / np.sqrt(n)
print("standard error: %s" % se)

mean = 65.41056475555557
std = 3.9682140600266522
5735
standard error: 0.052399649309995376


In [20]:
lcb = mean - 1.96 * se
ucb = mean + 1.96 * se
print((lcb, ucb))

(65.30786144290798, 65.51326806820316)


In [21]:
# convert the endpoints of the confidence interval from inches to back to cm
print(65.308 * 2.54)
print(65.513 * 2.54)

165.88232000000002
166.40302000000003


### Question 4
Partition the sample based on 10-year age bands, i.e. the resulting groups will consist of people with ages from 18-28, 29-38, etc. Construct 95% confidence intervals for the difference between the mean BMI for females and for males within each age band.

In [23]:
da['agegrp'] = pd.cut(da.RIDAGEYR, [18, 28, 38, 48, 58, 68, 80])
da.groupby(['agegrp', 'RIAGENDRx']).agg({'BMXBMI': [np.mean, np.std, np.size]})

BMXBMI                 
                         mean       std   size
agegrp   RIAGENDRx                            
(18, 28] Female     28.019433  8.048854  498.0
         Male       27.058186  6.679515  458.0
(28, 38] Female     29.943443  7.959097  494.0
         Male       29.697180  6.726690  467.0
(38, 48] Female     31.003733  8.044642  514.0
         Male       29.514646  6.104950  398.0
(48, 58] Female     30.787361  7.647590  454.0
         Male       29.385132  6.151534  419.0
(58, 68] Female     31.054664  7.779502  466.0
         Male       29.232462  5.959024  470.0
(68, 80] Female     29.412685  6.403343  480.0
         Male       28.255319  5.139608  484.0

In [24]:
# 95% confidence intervals for the difference btw the mean BMI for females and for males:
# i.e. 18-28
diff_mean = 28.019433 - 27.058186
se_female = 8.048854 / np.sqrt(498)
se_male = 6.679515 / np.sqrt(458)
se = np.sqrt(se_female ** 2 + se_male ** 2)
print(se)

0.4769728809100782


In [25]:
lcb = diff_mean - 1.96 * se
ucb = diff_mean + 1.96 * se
print((lcb, ucb))

(0.026380153416246888, 1.8961138465837535)


### Question 5
Construct a 95% confidence interval for the first BPXSY1 and second systolic blood pressure measures BPXSY2, and for the difference between the first and second systolic blood pressure measurements within a subject.

In [27]:
mean = da['BPXSY1'].mean()
print('mean = %s' % mean)

sd = da['BPXSY1'].std()
print('sd = %s' % sd)

n = len(da)
print('n = %s' % n)

se = sd / np.sqrt(n)
print('se = %s' % se)

mean = 125.08461396037771
sd = 18.480872651654824
n = 5735
se = 0.24403704821379846


In [28]:
lcb = mean - 1.96 * se
ucb = mean + 1.96 * se
print((lcb, ucb))

(124.60630134587866, 125.56292657487676)


### Question 6
Construct a 95% confidence interval for the mean difference between the average age of a smoker, and the average age of a non-smoker.

In [29]:
da['SMQ020'] = da.SMQ020.replace({1:'Yes', 2:'No', 7:np.nan, 9:np.nan})
dz = da[['SMQ020x', 'RIDAGEYR']].dropna()
dz.groupby('SMQ020x').agg({'RIDAGEYR': [np.mean, np.std, np.size]})

RIDAGEYR                 
              mean        std  size
SMQ020x                            
No       45.259836  18.543286  3406
Yes      52.096593  17.461141  2319

In [30]:
se_yes = 17.461141 / np.sqrt(2319)
se_no = 18.543286 / np.sqrt(3406)
diff_se = np.sqrt(se_yes ** 2 + se_no ** 2)
mean = 52.096593 - 45.259836
lcb = mean - 1.96 * diff_se
ucb = mean + 1.96 * diff_se
print((lcb, ucb))

(5.891820266365192, 7.781693733634805)
